In [6]:
import torch
import torch.nn as nn
import torchvision as vision
from torchvision.transforms import Lambda,Compose,ToTensor,Normalize,Grayscale
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Importing a dataset from Kaggle

1. Login to Kaggle
1. Click on your profile image and select account
1. Toward the middle of the page click on "create new API token". It will download a kaggle.json file
1. Upload that file when prompted in the section below

**Note**: make sure the name of the file matches the name in line 4 below

In [4]:
from google.colab import files
file=files.upload()
!mkdir -p /root/.kaggle
!mv kaggle.json  /root/.kaggle
!kaggle datasets download -d msambare/fer2013
!unzip fer2013.zip>/dev/null


Saving kaggle.json to kaggle.json
 81% 49.0M/60.3M [00:00<00:00, 95.0MB/s]
100% 60.3M/60.3M [00:00<00:00, 96.4MB/s]


## Reading the data

The dataset contains 7 classes ```['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']```
 
All images belonging to the same class are saved in the same directory (click on the folder icon on the left panel to explore)

When the images are organized in such a way the best method to read them is the ```ImageFolder```

In [9]:
# It is easier to deal with grayscale images
transform=Compose([Grayscale(),ToTensor()])
dataset_train=datasets.ImageFolder("train",transform=transform)
dataset_test=datasets.ImageFolder("test",transform=transform)

In [8]:
dataset_train.classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [11]:
print("There are {} images for training and {} for testing".format(len(dataset_train),len(dataset_test)))

There are 28709 images for training and 7178 for testing


In [12]:
# check the size of the images
itr=iter(dataset_train)
img,label=next(itr)
img.size()

torch.Size([1, 48, 48])

In [13]:
train_loader=DataLoader(dataset_train,batch_size=64,shuffle=True)
test_loader=DataLoader(dataset_test,batch_size=64,shuffle=False)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu=nn.ReLU()
        self.flatten=nn.Flatten()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)
        self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        self.pool1=nn.MaxPool2d(kernel_size=2)
        self.drop1=nn.Dropout()
        self.conv3=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3)
        self.conv4=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3)
        self.pool2=nn.MaxPool2d(kernel_size=2)
        self.drop2=nn.Dropout()
        self.fc1=nn.Linear(in_features=9*9*256,out_features=1024)
        self.drop3=nn.Dropout()
        self.fc2=nn.Linear(in_features=1024,out_features=7)
    def forward(self,x):
        #input (1,48,48)
        x=self.conv1(x)
        x=self.relu(x)
        #input (32,46,46)
        x=self.conv2(x)
        x=self.relu(x)
        #input (64,44,44)
        x=self.pool1(x)
        x=self.drop1(x)
        
        #input (64,22,22)
        x=self.conv3(x)
        x=self.relu(x)
        #input (128,20,20)
        x=self.conv4(x)
        x=self.relu(x)
        #input (256,18,18)
        x=self.pool2(x)
        x=self.drop2(x)
        #input (256,9,9)
        x=self.flatten(x)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.drop3(x)
        x=self.fc2(x)
        return x

In [15]:
%%time
import torch.optim as optim
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())
from tqdm import tqdm
epochs=10

for epoch in range(epochs):
    print(f"epoch {epoch}:",end=' ')
    train_loader=tqdm(train_loader)
   
    for imgs,labels in train_loader:
        imgs=imgs.cuda()
        labels=labels.cuda()
        outputs=model(imgs)
        optimizer.zero_grad()
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
    print(loss.item())
        

NameError: ignored

In [ ]:
total=len(dataset_test)
correct=0
for imgs,labels in loader_test:
    imgs=imgs.cuda()
    labels=labels.cuda()
    outputs=model(imgs)
    _,predicted=torch.max(outputs.data,1)
    tmp=(predicted==labels).sum()
    correct+=tmp
print(correct/total)